In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime, timedelta, date

In [2]:
requests = pd.read_csv('../data/hubNashville__311__Service_Requests.csv')
requests.head()

/var/folders/46/zm0dtt4s22d760_6ml4_t_vr0000gn/T/ipykernel_43952/1764631690.py:1: DtypeWarning: Columns (10,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  requests = pd.read_csv('../data/hubNashville__311__Service_Requests.csv')


,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,966190,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,05/04/2022 05:54:30 PM,05/04/2022 05:54:30 PM,Phone,NaN,NaN,...,False,NaN,True,1301 Pilgrim Dr,MADISON,8,37115.0,36.271880,-86.717549,"(36.2718801, -86.7175492)"
1,932934,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,04/01/2022 09:01:39 PM,04/05/2022 11:29:54 AM,Phone,NaN,NaN,...,False,NaN,False,1237 Tuckahoe Dr,NASHVILLE,3,37207.0,36.257270,-86.763690,"(36.25727, -86.76369)"
2,932939,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,04/01/2022 09:04:50 PM,04/06/2022 08:14:15 PM,Phone,NaN,NaN,...,False,NaN,False,3613 Wells Ct,ANTIOCH,33,37013.0,36.080968,-86.602406,"(36.080968, -86.602406)"
3,932935,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,04/01/2022 09:01:47 PM,04/01/2022 09:01:47 PM,Phone,NaN,NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,932938,Closed,Public Safety,Request Non-Emergency Police Assistance,NaN,04/01/2022 09:03:10 PM,04/01/2022 10:04:31 PM,Phone,NaN,NaN,...,False,NaN,False,401 Broadway,NASHVILLE,19,37203.0,36.160740,-86.777260,"(36.16074, -86.77726)"


In [3]:
requests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076715 entries, 0 to 1076714
Data columns (total 21 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Request #                   1076715 non-null  int64  
 1   Status                      1076715 non-null  object 
 2   Request Type                1076287 non-null  object 
 3   Subrequest Type             1071402 non-null  object 
 4   Additional Subrequest Type  1026917 non-null  object 
 5   Date / Time Opened          1076715 non-null  object 
 6   Date / Time Closed          1063766 non-null  object 
 7   Request Origin              1076715 non-null  object 
 8   System of Origin ID         39480 non-null    object 
 9   Contact Type                29999 non-null    object 
 10  Preferred Language          1 non-null        object 
 11  State Issue                 1076715 non-null  bool   
 12  Parent Request              10636 non-null    float64
 1

In [4]:
requests.groupby(['Request Type'])['Request #'].nunique().sort_values(ascending = False)

Request Type
Resolved by hubNashville on First Call             329817
Trash, Recycling & Litter                          321623
Streets, Roads & Sidewalks                         175561
Public Safety                                      102302
Property Violations                                 63331
COVID-19                                            22519
Other Metro Services and Forms                      18481
Electric & Water General                            14184
Transit                                             11886
Customer Experience Surveys                          4792
Parks                                                2873
Permits                                              2818
Hands On Volunteers                                  2639
Police Chief Search                                  1080
Reopening Safely                                      535
Planning & Zoning                                     519
Other                                                 467
T

In [5]:
requests[['Date Opened', 'Time Opened']] = requests['Date / Time Opened'].str.split(' ', 1, expand=True)
requests.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location,Date Opened,Time Opened
0,966190,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,05/04/2022 05:54:30 PM,05/04/2022 05:54:30 PM,Phone,NaN,NaN,...,True,1301 Pilgrim Dr,MADISON,8,37115.0,36.271880,-86.717549,"(36.2718801, -86.7175492)",05/04/2022,05:54:30 PM
1,932934,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,04/01/2022 09:01:39 PM,04/05/2022 11:29:54 AM,Phone,NaN,NaN,...,False,1237 Tuckahoe Dr,NASHVILLE,3,37207.0,36.257270,-86.763690,"(36.25727, -86.76369)",04/01/2022,09:01:39 PM
2,932939,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,04/01/2022 09:04:50 PM,04/06/2022 08:14:15 PM,Phone,NaN,NaN,...,False,3613 Wells Ct,ANTIOCH,33,37013.0,36.080968,-86.602406,"(36.080968, -86.602406)",04/01/2022,09:04:50 PM
3,932935,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,04/01/2022 09:01:47 PM,04/01/2022 09:01:47 PM,Phone,NaN,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/01/2022,09:01:47 PM
4,932938,Closed,Public Safety,Request Non-Emergency Police Assistance,NaN,04/01/2022 09:03:10 PM,04/01/2022 10:04:31 PM,Phone,NaN,NaN,...,False,401 Broadway,NASHVILLE,19,37203.0,36.160740,-86.777260,"(36.16074, -86.77726)",04/01/2022,09:03:10 PM


In [6]:
requests[['Date Closed', 'Time Closed']] = requests['Date / Time Closed'].str.split(' ', 1, expand=True)
requests.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,City,Council District,ZIP,Latitude,Longitude,Mapped Location,Date Opened,Time Opened,Date Closed,Time Closed
0,966190,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,05/04/2022 05:54:30 PM,05/04/2022 05:54:30 PM,Phone,NaN,NaN,...,MADISON,8,37115.0,36.271880,-86.717549,"(36.2718801, -86.7175492)",05/04/2022,05:54:30 PM,05/04/2022,05:54:30 PM
1,932934,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,04/01/2022 09:01:39 PM,04/05/2022 11:29:54 AM,Phone,NaN,NaN,...,NASHVILLE,3,37207.0,36.257270,-86.763690,"(36.25727, -86.76369)",04/01/2022,09:01:39 PM,04/05/2022,11:29:54 AM
2,932939,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,04/01/2022 09:04:50 PM,04/06/2022 08:14:15 PM,Phone,NaN,NaN,...,ANTIOCH,33,37013.0,36.080968,-86.602406,"(36.080968, -86.602406)",04/01/2022,09:04:50 PM,04/06/2022,08:14:15 PM
3,932935,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,04/01/2022 09:01:47 PM,04/01/2022 09:01:47 PM,Phone,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,04/01/2022,09:01:47 PM,04/01/2022,09:01:47 PM
4,932938,Closed,Public Safety,Request Non-Emergency Police Assistance,NaN,04/01/2022 09:03:10 PM,04/01/2022 10:04:31 PM,Phone,NaN,NaN,...,NASHVILLE,19,37203.0,36.160740,-86.777260,"(36.16074, -86.77726)",04/01/2022,09:03:10 PM,04/01/2022,10:04:31 PM


In [13]:
requests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076715 entries, 0 to 1076714
Data columns (total 25 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Request #                   1076715 non-null  int64  
 1   Status                      1076715 non-null  object 
 2   Request Type                1076287 non-null  object 
 3   Subrequest Type             1071402 non-null  object 
 4   Additional Subrequest Type  1026917 non-null  object 
 5   Date / Time Opened          1076715 non-null  object 
 6   Date / Time Closed          1063766 non-null  object 
 7   Request Origin              1076715 non-null  object 
 8   System of Origin ID         39480 non-null    object 
 9   Contact Type                29999 non-null    object 
 10  Preferred Language          1 non-null        object 
 11  State Issue                 1076715 non-null  bool   
 12  Parent Request              10636 non-null    float64
 1

In [16]:
requests['Date Opened'] = requests['Date Opened'].astype('datetime64[ns]')
requests['Date Closed'] = requests['Date Closed'].astype('datetime64[ns]')
requests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076715 entries, 0 to 1076714
Data columns (total 25 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   Request #                   1076715 non-null  int64         
 1   Status                      1076715 non-null  object        
 2   Request Type                1076287 non-null  object        
 3   Subrequest Type             1071402 non-null  object        
 4   Additional Subrequest Type  1026917 non-null  object        
 5   Date / Time Opened          1076715 non-null  object        
 6   Date / Time Closed          1063766 non-null  object        
 7   Request Origin              1076715 non-null  object        
 8   System of Origin ID         39480 non-null    object        
 9   Contact Type                29999 non-null    object        
 10  Preferred Language          1 non-null        object        
 11  State Issue             

In [17]:
requests['Days to Resolve'] = requests['Date Closed'] - requests['Date Opened']
requests.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,Council District,ZIP,Latitude,Longitude,Mapped Location,Date Opened,Time Opened,Date Closed,Time Closed,Days to Resolve
0,966190,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,05/04/2022 05:54:30 PM,05/04/2022 05:54:30 PM,Phone,NaN,NaN,...,8,37115.0,36.271880,-86.717549,"(36.2718801, -86.7175492)",2022-05-04,05:54:30 PM,2022-05-04,05:54:30 PM,0 days
1,932934,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,04/01/2022 09:01:39 PM,04/05/2022 11:29:54 AM,Phone,NaN,NaN,...,3,37207.0,36.257270,-86.763690,"(36.25727, -86.76369)",2022-04-01,09:01:39 PM,2022-04-05,11:29:54 AM,4 days
2,932939,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,04/01/2022 09:04:50 PM,04/06/2022 08:14:15 PM,Phone,NaN,NaN,...,33,37013.0,36.080968,-86.602406,"(36.080968, -86.602406)",2022-04-01,09:04:50 PM,2022-04-06,08:14:15 PM,5 days
3,932935,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,04/01/2022 09:01:47 PM,04/01/2022 09:01:47 PM,Phone,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-04-01,09:01:47 PM,2022-04-01,09:01:47 PM,0 days
4,932938,Closed,Public Safety,Request Non-Emergency Police Assistance,NaN,04/01/2022 09:03:10 PM,04/01/2022 10:04:31 PM,Phone,NaN,NaN,...,19,37203.0,36.160740,-86.777260,"(36.16074, -86.77726)",2022-04-01,09:03:10 PM,2022-04-01,10:04:31 PM,0 days


In [18]:
requests['Time Opened'] = requests['Time Opened'].apply(pd.to_timedelta)
requests['Time Closed'] = requests['Time Closed'].apply(pd.to_timedelta)
requests.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,Council District,ZIP,Latitude,Longitude,Mapped Location,Date Opened,Time Opened,Date Closed,Time Closed,Days to Resolve
0,966190,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,05/04/2022 05:54:30 PM,05/04/2022 05:54:30 PM,Phone,NaN,NaN,...,8,37115.0,36.271880,-86.717549,"(36.2718801, -86.7175492)",2022-05-04,0 days 05:54:30,2022-05-04,0 days 05:54:30,0 days
1,932934,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,04/01/2022 09:01:39 PM,04/05/2022 11:29:54 AM,Phone,NaN,NaN,...,3,37207.0,36.257270,-86.763690,"(36.25727, -86.76369)",2022-04-01,0 days 09:01:39,2022-04-05,0 days 11:29:54,4 days
2,932939,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,04/01/2022 09:04:50 PM,04/06/2022 08:14:15 PM,Phone,NaN,NaN,...,33,37013.0,36.080968,-86.602406,"(36.080968, -86.602406)",2022-04-01,0 days 09:04:50,2022-04-06,0 days 08:14:15,5 days
3,932935,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,04/01/2022 09:01:47 PM,04/01/2022 09:01:47 PM,Phone,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-04-01,0 days 09:01:47,2022-04-01,0 days 09:01:47,0 days
4,932938,Closed,Public Safety,Request Non-Emergency Police Assistance,NaN,04/01/2022 09:03:10 PM,04/01/2022 10:04:31 PM,Phone,NaN,NaN,...,19,37203.0,36.160740,-86.777260,"(36.16074, -86.77726)",2022-04-01,0 days 09:03:10,2022-04-01,0 days 10:04:31,0 days


In [19]:
requests['Time to Resolve'] = requests['Time Closed'] - requests['Time Opened']
requests.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,ZIP,Latitude,Longitude,Mapped Location,Date Opened,Time Opened,Date Closed,Time Closed,Days to Resolve,Time to Resolve
0,966190,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,05/04/2022 05:54:30 PM,05/04/2022 05:54:30 PM,Phone,NaN,NaN,...,37115.0,36.271880,-86.717549,"(36.2718801, -86.7175492)",2022-05-04,0 days 05:54:30,2022-05-04,0 days 05:54:30,0 days,0 days 00:00:00
1,932934,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,04/01/2022 09:01:39 PM,04/05/2022 11:29:54 AM,Phone,NaN,NaN,...,37207.0,36.257270,-86.763690,"(36.25727, -86.76369)",2022-04-01,0 days 09:01:39,2022-04-05,0 days 11:29:54,4 days,0 days 02:28:15
2,932939,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,04/01/2022 09:04:50 PM,04/06/2022 08:14:15 PM,Phone,NaN,NaN,...,37013.0,36.080968,-86.602406,"(36.080968, -86.602406)",2022-04-01,0 days 09:04:50,2022-04-06,0 days 08:14:15,5 days,-1 days +23:09:25
3,932935,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,04/01/2022 09:01:47 PM,04/01/2022 09:01:47 PM,Phone,NaN,NaN,...,NaN,NaN,NaN,NaN,2022-04-01,0 days 09:01:47,2022-04-01,0 days 09:01:47,0 days,0 days 00:00:00
4,932938,Closed,Public Safety,Request Non-Emergency Police Assistance,NaN,04/01/2022 09:03:10 PM,04/01/2022 10:04:31 PM,Phone,NaN,NaN,...,37203.0,36.160740,-86.777260,"(36.16074, -86.77726)",2022-04-01,0 days 09:03:10,2022-04-01,0 days 10:04:31,0 days,0 days 01:01:21
